# plotting eQTLs

### Kynon Jade Benjamin and Apuã Paquola

In [ ]:
import re
import functools
import subprocess
import numpy as np
import pandas as pd
from plotnine import *
from pandas_plink import read_plink
from warnings import filterwarnings
from matplotlib.cbook import mplDeprecation

filterwarnings("ignore",category=mplDeprecation)
filterwarnings('ignore', category=UserWarning, module='plotnine.*')
filterwarnings('ignore', category=DeprecationWarning, module='plotnine.*')

In [ ]:
config = {
    #'de_file': '/ceph/projects/v2_phase3_paper/inputs/differentialExpression/_m/genes/diffExpr_szVctl_full.txt',
    'biomart_file': '../_h/biomart.csv',
    'residual_expression_file': '../../_m/genes_residualized_expression.csv',
    'phenotype_file': '/ceph/projects/v3_phase3_paper/inputs/phenotypes/merged/_m/merged_phenotypes.csv',
    'plink_file_prefix': '/ceph/projects/v3_phase3_paper/inputs/genotypes/to_brnum/phase3_plink/_m/phase3',
    'matrixeqtl_output_file': '../../../_m/cis_eqtls_genes.ctxt',
    'gwas_snp_file': '/ceph/projects/v3_phase3_paper/inputs/gwas/PGC2_CLOZUK/map_phase3/_m/libd_hg38_pggc2sz_snps.tsv'
}


In [ ]:
@functools.lru_cache()
def get_biomart_df():
    biomart = pd.read_csv(config['biomart_file'])
    biomart['description'] = biomart['description'].str.replace('\[Source.*$','')
    return biomart


@functools.lru_cache()
def get_residual_expression_df():
    residual_expression_df = pd.read_csv(config['residual_expression_file'], index_col=0).transpose()
    return residual_expression_df


@functools.lru_cache()
def get_pheno_df():
    pheno_df = pd.read_csv(config['phenotype_file'], index_col=0)
    return pheno_df


@functools.lru_cache()
def get_expression_and_pheno_df():
    return pd.merge(get_pheno_df(), get_residual_expression_df(), left_index=True, right_index=True)


@functools.lru_cache()
def get_de_df(filename):
    de_df = pd.read_csv(filename, sep='\t', index_col=0, usecols=[0]+list(range(5,13)))
    return de_df


def get_gene_symbol(gene_id, biomart=get_biomart_df()):
    ensge = re.sub('\..+$','', gene_id)
    ggg = biomart[biomart['ensembl_gene_id']==ensge]
    if ggg.shape[0]==0:
        return '', ''
    gs = ggg['external_gene_name'].values[0]
    de = ggg['description'].values[0]
    if type(de)!=str:
        de = ''
    de = re.sub('\[Source:.*$','',de)
    return gs, de



@functools.lru_cache()
def get_gene_id_df():
    return pd.DataFrame({'gene_id': get_residual_expression_df().columns,
                         'ensembl_gene_id': get_residual_expression_df().columns.str.replace('\..+$','')})


@functools.lru_cache()
def gene_info_from_symbol(gene_symbol):
    gene_id_df = get_gene_id_df()
    biomart=get_biomart_df()
    r = biomart[biomart['external_gene_name']==gene_symbol].merge(gene_id_df, on='ensembl_gene_id', how='left')
    return r


@functools.lru_cache()
def gene_id_from_symbol(gene_symbol):
    df = gene_info_from_symbol(gene_symbol)
    assert df.shape[0] == 1
    return df[['gene_id']].iloc[0].values[0]


@functools.lru_cache()
def get_plink_tuple():
    '''
    Usage: (bim, fam, bed) = get_plink_tuple()
    '''
    return read_plink(config['plink_file_prefix'])


@functools.lru_cache()
def get_eqtl_df(fdr=0.05):
    with subprocess.Popen('''awk ' ($6<%f) || (NR==1) {print}' %s ''' % 
                          (fdr, config['matrixeqtl_output_file']),
                          shell=True, stdout=subprocess.PIPE) as p:
        eqtl_df = pd.read_csv(p.stdout, sep='\t')
    return eqtl_df


@functools.lru_cache()
def get_gwas_snps():
    return pd.read_csv(config['gwas_snp_file'], sep='\t', index_col=0)
 
    
def get_gwas_snp(snp_id):
    gwas = get_gwas_snps()
    r = gwas[gwas['our_snp_id']==snp_id]
    assert len(r) == 1
    return r


In [ ]:
def letter_snp(number, a0, a1):
    '''
    Example:
    letter_snp(0, 'A', 'G') is 'AA'
    letter_snp(1, 'A', 'G') is 'AG'
    letter_snp(2, 'A', 'G') is 'GG'
    
    '''
    if np.isnan(number):
        return np.nan
    if len(a0)==1 and len(a1)==1:
        sep = ''
    else:
        sep = ' '
    return sep.join(sorted([a0]*int(number) + [a1]*(2-int(number))))


In [ ]:
@functools.lru_cache()
def get_snp_df(snp_id):
    '''
    Returns a dataframe containing the genotype on snp snp_id.
    The allele count is the same as in the plink files.
    
    Example: 
    get_snp_df('rs653953').head(5)
    
            rs653953_num rs653953_letter rs653953
    Br5168             0              GG    0\nGG
    Br2582             1              AG    1\nAG
    Br2378             1              AG    1\nAG
    Br5155             2              AA    2\nAA
    Br5182             2              AA    2\nAA
    '''
    (bim, fam, bed) = get_plink_tuple()
    brain_ids = list(set(get_expression_and_pheno_df()['BrNum']).intersection(set(fam['fid'])))
    snp_info = bim[bim['snp']==snp_id]
    snp_pos = snp_info.iloc[0]['i']
    fam_pos = list(fam.set_index('fid').loc[brain_ids]['i'])
    dfsnp = (pd.DataFrame(bed[[snp_pos]].compute()[:,fam_pos], 
                          columns=brain_ids, index=[snp_id + '_num'])
             .transpose().dropna())
    my_letter_snp = functools.partial(letter_snp, a0=snp_info.iloc[0]['a0'], a1=snp_info.iloc[0]['a1'])
    # the 2 - in next line is to workarount a possible bug in pandas_plink? a1 and a0 inverted
    dfsnp[[snp_id + '_num']] = 2 - dfsnp[[snp_id + '_num']].astype('int')
    dfsnp[snp_id + '_letter'] = dfsnp[snp_id + '_num'].apply(my_letter_snp)
    dfsnp[snp_id] = (dfsnp[snp_id + '_num'].astype('str') + '\n' + 
                     dfsnp[snp_id + '_letter'].astype('str')).astype('category')
    return dfsnp

In [ ]:
@functools.lru_cache()
def get_gwas_ordered_snp_df(snp_id):
    '''
    Returns a dataframe containing the genotype on snp snp_id.
    The allele count is the number of risk alleles according to GWAS.
    
    Example: 
    get_gwas_ordered_snp_df('rs653953').head(5)
    
            rs653953_num rs653953_letter rs653953
    Br5168             2              GG    2\nGG
    Br2582             1              AG    1\nAG
    Br2378             1              AG    1\nAG
    Br5155             0              AA    0\nAA
    Br5182             0              AA    0\nAA
    '''
    pgc = get_gwas_snps()
    dfsnp = get_snp_df(snp_id).copy()
    gwas_snp = get_gwas_snp(snp_id)
    if gwas_snp['pgc2_a1_same_as_our_counted'].iloc[0]:
        if gwas_snp['OR'].iloc[0] > 1:
            pass
        else:
            dfsnp[[snp_id + '_num']] = 2 - dfsnp[[snp_id + '_num']]
    else:
        if gwas_snp['OR'].iloc[0] > 1:
            dfsnp[[snp_id + '_num']] = 2 - dfsnp[[snp_id + '_num']]
        else:
            pass
    dfsnp[snp_id] = (dfsnp[snp_id + '_num'].astype('str') + '\n' + 
                     dfsnp[snp_id + '_letter'].astype('str')).astype('category')
    
    return dfsnp

In [ ]:
@functools.lru_cache()
def get_risk_allele(snp_id):
    gwas_snp = get_gwas_snp(snp_id)
    if gwas_snp['OR'].iloc[0] > 1:
        ra = gwas_snp['A1'].iloc[0]
    else:
        ra = gwas_snp['A2'].iloc[0]
    return ra

In [ ]:
def get_snp_gene_pheno_df(snp_id, gene_id, snp_df_func):
    pheno_columns = list(get_pheno_df().columns)
    expr_df = get_expression_and_pheno_df()[pheno_columns + [gene_id]]
    snp_df =  snp_df_func(snp_id)
    return expr_df.merge(snp_df, left_on='BrNum', right_index=True)
    

def simple_snp_expression_plot_impl(snp_id, gene_id, snp_df_func):
    df = get_snp_gene_pheno_df(snp_id, gene_id, snp_df_func)
    y0 = df[gene_id].quantile(.01) - 0.26
    y1 = df[gene_id].quantile(.99) + 0.26
    p = ggplot(df, aes(x=snp_id, y=gene_id)) \
    + geom_boxplot(fill='red', alpha=0.4, outlier_alpha=0) \
    + geom_jitter(width=0.25, stroke=0, alpha=0.6) + ylim(y0, y1) \
    + theme_matplotlib()\
    + theme(axis_text=element_text(size=18), 
            axis_title=element_text(size=21), 
            plot_title=element_text(size=22))
    return p
    

def simple_snp_expression_plot(snp_id, gene_id):
    return simple_snp_expression_plot_impl(snp_id, gene_id, get_snp_df)


def simple_gwas_ordered_snp_expression_plot(snp_id, gene_id):
    return simple_snp_expression_plot_impl(snp_id, gene_id, get_gwas_ordered_snp_df)


def simple_snp_expression_pheno_plot_impl(snp_id, gene_id, snp_df_func, pheno_var):
    df = get_snp_gene_pheno_df(snp_id, gene_id, snp_df_func)
    y0 = df[gene_id].quantile(.01) - 0.26
    y1 = df[gene_id].quantile(.99) + 0.26
    pjd = position_jitterdodge(jitter_width=0.22)
    p = ggplot(df, aes(x=snp_id, y=gene_id, fill=pheno_var)) \
    + geom_boxplot(alpha=0.4, outlier_alpha=0) \
    + geom_jitter(position=pjd, stroke=0, alpha=0.6) + ylim(y0, y1) \
    + theme_matplotlib()\
    + theme(axis_text=element_text(size=18), 
            axis_title=element_text(size=21), 
            plot_title=element_text(size=22), 
            legend_text=element_text(size=14))
    return p


In [ ]:
def gwas_annotation(snp_id):
    return 'SZ GWAS pvalue: %.1e' % get_gwas_snp(snp_id).iloc[0]['P']


def eqtl_interaction_annotation(snp_id, gene_id):
    eqtl_df = get_eqtl_df()
    r = eqtl_df[(eqtl_df['SNP']==snp_id) & (eqtl_df['gene']==gene_id)]
    assert len(r)==1
    return 'genotype:brain region eQTL FDR: %.1e' % r.iloc[0]['FDR']


def risk_allele_annotation(snp_id):
    return 'SZ risk allele: %s' % get_risk_allele(snp_id)


def annotated_genotype_brain_region_interaction_eqtl_plot(snp_id, gene_id):
    p = simple_snp_expression_pheno_plot_impl(snp_id, gene_id,
                                              get_snp_df, 'Region')
    gene_symbol, gene_description = get_gene_symbol(gene_id)
    title ="\n".join([gene_symbol,
                      eqtl_interaction_annotation(snp_id, gene_id)])
    p += ggtitle(title) + ylab('Residualized expression')
    return p


def gwas_annotated_genotype_brain_region_interaction_eqtl_plot(snp_id, gene_id):
    p = simple_snp_expression_pheno_plot_impl(snp_id, gene_id, 
                                              get_gwas_ordered_snp_df, 'Region')
    gene_symbol, gene_description = get_gene_symbol(gene_id)
    title ="\n".join([gene_symbol,
                      risk_allele_annotation(snp_id),
                      gwas_annotation(snp_id),     
                      eqtl_interaction_annotation(snp_id, gene_id)])
    p += ggtitle(title) + ylab('Residualized expression')
    return p


def save_plot(p, fn):
    for ext in ['png', 'pdf', 'svg']:
        p.save(fn + '.' + ext)
    

# Top 10 eQTLs

In [ ]:
eqtl_df = get_eqtl_df()
eqtl_df.head()

In [ ]:
top_10 = eqtl_df.sort_values('p-value', ascending=True).groupby('gene').first().sort_values('p-value').head(10).reset_index()
for x in top_10.itertuples():
    filename = "top_%d_interaction_eqtl_caudate_hippo" % x.Index
    p = annotated_genotype_brain_region_interaction_eqtl_plot(x.SNP, x.gene)
    print(filename, x.Index, x.SNP, x.gene)
    print(p)
    save_plot(p, filename)
    

# Top 10 eQTLs with GWAS significant index SNP

In [ ]:
gwas_eqtl_df = eqtl_df.merge(get_gwas_snps(), left_on = 'SNP', right_on = 'our_snp_id', suffixes=['','_gwas'])
print(gwas_eqtl_df.shape)
gwas_eqtl_df.head()

In [ ]:
top_gwas_eqtl_df = gwas_eqtl_df[(gwas_eqtl_df['P']<5e-8) & (gwas_eqtl_df['is_index_snp'])].sort_values(['FDR', 'P'])
print(top_gwas_eqtl_df.shape)
top_gwas_eqtl_df.head()

In [ ]:
top10 = top_gwas_eqtl_df.sort_values('p-value', ascending=True).groupby('gene').first().sort_values('p-value').head(10).reset_index()
for x  in top10.itertuples():
    filename = "top_%d_caudate_hippo_interaction_eqtl_in_gwas_significant_index_snps" % x.Index
    p = gwas_annotated_genotype_brain_region_interaction_eqtl_plot(x.SNP, x.gene)
    print(filename, x.Index, x.SNP, x.gene)
    print(p)
    save_plot(p, filename)

# eQTL for top GWAS index SNP

In [ ]:
a = top_gwas_eqtl_df.sort_values(['P', 'FDR'])
p = gwas_annotated_genotype_brain_region_interaction_eqtl_plot(a.iloc[0]['SNP'], a.iloc[0]['gene'])
save_plot(p, 'caudate_hippo_interaction_eqtl_for_top_index_snp')
p